In [ ]:
import csv
import pandas as pd

## File Merge
### Merge scopus and web of science (WoS)

In [ ]:
t1 = pd.read_csv('output_audit/scopus.csv', header=None)
t2 = pd.read_csv('output_audit/wos.csv', header=None)

scopus = t1.values.tolist()
wos = t2.values.tolist()

with open('output_audit/merge.csv', 'w', newline='', encoding='utf-8-sig') as outFile, open('output_old/remove.csv', 'w',
                                                                                newline='',
                                                                                encoding='utf-8-sig') as removeFile:

    line = ['Authors', 'Author Full Names', 'Title', 'Source Title', 'Document Type', 'Publisher',
            'Publication Year', 'Volume', 'Issue', 'DOI', 'Link', 'ISSN', 'ISBN', 'Number of Pages',
            'Author Keywords', 'Language', 'Source', 'Abstract']
    csv_writer = csv.writer(outFile)
    csv_writer.writerow(line)

    nanKey_num = 0  # number of nan keywords
    remove_page = 0  # number of removed papers in page
    remove_paper = 0  # number of removed papers in paper

    # scopus
    for s in scopus[1:]:
        page = s[10]
        document = s[28]
        publisher = s[21]
        source_title = s[4]
        keywords = s[17]

        newline = [s[0], s[15], s[2], source_title, document, publisher, s[3], s[5], s[6], s[12], s[13], s[22], s[23], page,
                    keywords, s[31], s[36], s[16]]

        # Update the keywords
        if str(keywords) == '' or str(keywords) == 'nan':
            keywords = s[18]
        # Write to Output file
        csv_writer = csv.writer(outFile)
        csv_writer.writerow(newline)

    # web of science
    for w in wos[1:]:
        page = w[52]
        document = w[13]
        publisher = w[37]
        source_title = w[9]
        keywords = w[19]

        newline = [w[1], w[5], w[8], source_title, document, publisher, w[46], w[47], w[48], w[56], w[57], w[40], w[42],
                    page, keywords, w[12], 'Web of Science', w[21]]

        # # Delete pages < 4
        # if delete_pages(page):
        #     remove_page += 1
        #     continue
        #
        # # Delete abnormal journals
        # if str(document) == 'Article':
        #     if delete_papers(publisher, source_title):
        #         remove_paper += 1
        #         # write the removed paper to another file
        #         csv_writer = csv.writer(removeFile)
        #         csv_writer.writerow(newline)
        #         continue

        # Update the keywords
        if str(keywords) == '' or str(keywords) == 'nan':
            keywords = w[20]
        # Write to Output file
        csv_writer = csv.writer(outFile)
        csv_writer.writerow(newline)

outFile.close()

## Find duplicate files
### To keep papers after removing duplicate in scopus and WoS

In [ ]:
merge = pd.read_csv('output_audit/merge.csv')
# find duplicates
merge[merge.duplicated(['Title'])].to_csv('output_audit/scopus+wos_duplicate.csv', index=False)
# store remaining
merge.drop_duplicates(['Title']).to_csv('output_audit/merge_duplicate_rm.csv', index=False)

## Remove unqualified papers
### 1. Non-English
### 2. Predatory journals or conferences
### 3. Small pages (< 5)
### 4. Outdated (year < 2015)

In [ ]:
def delete_pages(page):
    if str(page) != '#VALUE!' and str(page).isdigit():
        if 4 >= int(page) > 1:
            return True
    return False


def delete_papers(publisher, source_title):
    for p in predatory:
        paper = str(p)
        # if str(publisher) in paper or paper in str(publisher) or paper in str(source_title):
        #     return True
        if paper == publisher or paper == source_title:
            return True
    return False

t = pd.read_csv('output_audit/merge_duplicate_nonEnglish_rm.csv', header=None)
merge = t.values.tolist()

predatory = pd.read_csv('output_audit/Predatory.csv', header=None).values.tolist()

with open('output_audit/merge_exclusion_applied.csv', 'w', newline='', encoding='utf-8-sig') as outFile, \
        open('output_audit/remove_predatory.csv', 'w', newline='', encoding='utf-8-sig') as removeFilePred, \
        open('output_audit/remove_page.csv', 'w', newline='', encoding='utf-8-sig') as removeFilePage, \
        open('output_audit/remove_outdated.csv', 'w', newline='', encoding='utf-8-sig') as removeFileOut, \
        open('output_audit/remove_nonEn.csv', 'w', newline='', encoding='utf-8-sig') as removeFileNonEn:
    csv_writer = csv.writer(outFile)
    csv_writer.writerow(merge[0])

    remove_paper = 0  # number of removed papers in paper
    remove_page = 0  # number of removed papers in page
    remove_outdated = 0 # number of removed papers in outdated paper
    remove_nonEnglish = 0
    remain_paper = 0

    for m in merge[1:]:

        # Delete abnormal papers
        if delete_papers(m[5], m[3]):
            remove_paper += 1
            # write the removed paper to another file
            csv_writer = csv.writer(removeFilePred)
            csv_writer.writerow(m)
            continue

        # Delete pages <= 4
        if delete_pages(m[13]):
            remove_page += 1
            # write the removed paper to another file
            csv_writer = csv.writer(removeFilePage)
            csv_writer.writerow(m)
            continue
        if str(m[13]).isdigit():
            if int(m[13]) == 1:
                m[13] = 'nan'

        # Delete the outdated paper
        if str(m[6]).isdigit():
            if int(m[6]) > 2022 or int(m[6]) < 2015:
                remove_outdated += 1
                csv_writer = csv.writer(removeFileOut)
                csv_writer.writerow(m)
                continue
        else:
            remove_outdated += 1
            csv_writer = csv.writer(removeFileOut)
            csv_writer.writerow(m)
            continue

        # Delete non-English
        if str(m[15]) != "English":
            remove_nonEnglish += 1
            csv_writer = csv.writer(removeFileNonEn)
            csv_writer.writerow(m)
            continue

        # Write to Output file
        csv_writer = csv.writer(outFile)
        csv_writer.writerow(m)
        remain_paper += 1

    print("The number of removed papers in paper is %d" % remove_paper)
    print("The number of removed papers in page is %d" % remove_page)
    print("The number of removed papers in outdated is %d" % remove_outdated)
    print("The number of removed papers in non-English is %d" % remove_nonEnglish)
    print("The number of remaining papers is %d" % remain_paper)

## Add ScienceDirect Papers
### Merge the remaining papers of scopus and WoS with ScienceDirect

In [ ]:
df1 = pd.read_csv('output_audit/ScienceDirect(title-abstract).csv', header=None)
df2 = pd.read_csv('output_audit/merge_exclusion_applied.csv', header=None)

science_direct = df1.values.tolist()
origin = df2.values.tolist()

with open('output_audit/merge_exclusion+scienceDirect.csv', 'w', newline='', encoding='utf-8-sig') as outFile, \
        open('output_old/remove_sd_duplicate.csv', 'w', newline='', encoding='utf-8-sig') as removeFile:
    line = ['Authors', 'Author Full Names', 'Title', 'Source Title', 'Document Type', 'Publisher',
            'Publication Year', 'Volume', 'Issue', 'DOI', 'Link', 'ISSN', 'ISBN', 'Number of Pages',
            'Author Keywords', 'Language', 'Source', 'Abstract']
    csv_writer = csv.writer(outFile)
    csv_writer.writerow(line)

    for o in origin[1:]:
        csv_writer = csv.writer(outFile)
        csv_writer.writerow(o)

    for sd in science_direct[1:]:
        newline = [sd[1], sd[1], sd[3], sd[4], sd[0], sd[4], sd[5], sd[6], sd[7], sd[16], sd[15], sd[14], sd[13],
                    sd[8], sd[18], 'English', 'ScienceDirect', sd[17]]
        csv_writer = csv.writer(outFile)
        csv_writer.writerow(newline)

outFile.close()

## Find duplicate with ScienceDirect
### To keep papers after removing duplicate in scopus, WoS and ScienceDirect

In [ ]:
merge_sd = pd.read_csv('output_audit/merge_exclusion+scienceDirect.csv')
merge_sd[merge_sd.duplicated(['Title'])].to_csv('output_audit/afterScienceDirect/scienceDirect_duplicate.csv', index=False)
merge_sd.drop_duplicates(['Title']).to_csv('output_audit/afterScienceDirect/scienceDirect_duplicate_rm.csv', index=False)

## Remove unqualified papers with ScienceDirect
### 1. Non-English
### 2. Predatory journals or conferences
### 3. Small pages (< 5)
### 4. Outdated (year < 2015)

In [ ]:
t = pd.read_csv('output_audit/afterScienceDirect/scienceDirect_duplicate_rm.csv', header=None)
merge = t.values.tolist()

predatory = pd.read_csv('output_audit/Predatory.csv', header=None).values.tolist()

with open('output_audit/afterScienceDirect/merge_exclusion_applied.csv', 'w', newline='', encoding='utf-8-sig') as outFile, \
        open('output_audit/afterScienceDirect/remove_predatory.csv', 'w', newline='', encoding='utf-8-sig') as removeFilePred, \
        open('output_audit/afterScienceDirect/remove_page.csv', 'w', newline='', encoding='utf-8-sig') as removeFilePage, \
        open('output_audit/afterScienceDirect/remove_outdated.csv', 'w', newline='', encoding='utf-8-sig') as removeFileOut, \
        open('output_audit/afterScienceDirect/remove_nonEn.csv', 'w', newline='', encoding='utf-8-sig') as removeFileNonEn:
    csv_writer = csv.writer(outFile)
    csv_writer.writerow(merge[0])

    remove_paper = 0  # number of removed papers in paper
    remove_page = 0  # number of removed papers in page
    remove_outdated = 0 # number of removed papers in outdated paper
    remove_nonEnglish = 0
    remain_paper = 0

    for m in merge[1:]:

        # Delete abnormal papers
        if delete_papers(m[5], m[3]):
            remove_paper += 1
            # write the removed paper to another file
            csv_writer = csv.writer(removeFilePred)
            csv_writer.writerow(m)
            continue

        # Delete pages <= 4
        if delete_pages(m[13]):
            remove_page += 1
            # write the removed paper to another file
            csv_writer = csv.writer(removeFilePage)
            csv_writer.writerow(m)
            continue
        if str(m[13]).isdigit():
            if int(m[13]) == 1:
                m[13] = 'nan'

        # Delete the outdated paper
        if str(m[6]).isdigit():
            if int(m[6]) > 2022 or int(m[6]) < 2015:
                remove_outdated += 1
                csv_writer = csv.writer(removeFileOut)
                csv_writer.writerow(m)
                continue
        else:
            remove_outdated += 1
            csv_writer = csv.writer(removeFileOut)
            csv_writer.writerow(m)
            continue

        # Delete non-English
        if str(m[15]) != "English":
            remove_nonEnglish += 1
            csv_writer = csv.writer(removeFileNonEn)
            csv_writer.writerow(m)
            continue

        # Write to Output file
        csv_writer = csv.writer(outFile)
        csv_writer.writerow(m)
        remain_paper += 1

    print("The number of removed papers in paper is %d" % remove_paper)
    print("The number of removed papers in page is %d" % remove_page)
    print("The number of removed papers in outdated is %d" % remove_outdated)
    print("The number of removed papers in non-English is %d" % remove_nonEnglish)
    print("The number of remaining papers is %d" % remain_paper)